# **Chapter 10: Integrated Models**

## ***Learning Objectives:***

* Implement dynamic time-course simulations using FBA
* Integrate transcriptional regulation with metabolic network modeling

* Understand how integrating heterogeneous modeling approaches enables whole-cell models

We’re going to end this book with a topic that I’ve been obsessed with for almost 15
years now: the quest to build “whole-cell” models that consider every known gene and
molecule. This obsession began just as I started graduate school, when I read an article in
*The New York Times* (see Wade in Recommended Reading) that quoted Clyde Hutchison,
who studies *Mycoplasma*, as saying:

>The ultimate test of understanding a simple cell, more than being able to build
one, would be to build a computer model of the cell, because that really requires
understanding at a deeper level [emphasis mine].

The ultimate test! Hardly a day goes by when I don’t think about that quote; it became
my white whale, and I hope by the end of this book that some of you will also want to
join the chase.

In 1999, when that article caught my eye, decades’ worth of impressive efforts had
already gone toward whole-cell modeling. Francis Crick and Sydney Brenner had been
talking about “the complete solution of E. coli” as part of what they called “Project K”
(the “K” stands for the “K-12” E. coli lab strain) as early as 1973 in Cambridge, England.
In the 1980s, Michael Shuler at Cornell University made the first effort to model the
major biological processes in E. coli. At that time, very little was known about E. coli’s
genes, and so Shuler built an ODE-based model that was relatively focused on pathways
and processes as a whole (see Domach et al. in Recommended Reading), adding detail as
the years went by (see Shuler et al. in Recommended Reading). Around the same time,
Harold Morowitz at Yale University was advocating *Mycoplasma*, the simplest known
genus of free-living bacteria, as the best system for a comprehensive mathematical model
(see Morowitz in Recommended Reading). In the 1990s, Masaru Tomita and his
colleagues from Keio University in Japan developed a more detailed and gene-based
model called “E-Cell” (see Tomita et al. in Recommended Reading). And of course,
Bernhard Palsson at the University of Michigan, and later UCSD, was raising metabolic
modeling to new heights with the development of FBA (see Savinell and Palsson in
Recommended Reading). These and other key efforts all inspired me.

Looking over this impressive body of work, it became clear that no single modeling
approach – ODEs, stochastic simulations, or FBA – would be sufficient to model a whole
cell. In practical terms, it didn’t seem possible to use the same approaches to model, for
example, the progress of the DNA replication apparatus and the fluxes through the metabolic network. These processes are just too different, both in terms of their
underlying physical mechanisms as well as our ability to describe each of them.

As a result, some people began to focus on bringing different methods together to form
integrated models. At the time, this approach seemed to offer some major advantages if
it actually worked; you could divide the total functionality of the cell into pieces, model
each piece with the best available method, and then put the pieces together into one
whole model. Implementing this approach turned out to be a significant advance, and led
to the construction of the first whole-cell model.

I can now finally reveal my nefarious little plan now – to train you all to be whole-cell
modelers! Throughout this book we’ve addressed many of the key approaches you’ll
need to model various functions in a cell, and now we’ll go through the first critical steps
for putting them all together.

## **Section 10.1 Dynamic FBA: external vs. internal concentrations**

Let’s start with using FBA to run dynamic simulations. Wait, didn’t we say in Chapter 9
that FBA couldn’t be used to simulate dynamics? Isn’t one of the critical assumptions of
FBA that the metabolic network is at a steady state?


Well, yes and no. Imagine that you are growing a culture of *E. coli *in the lab. You have a flask containing glucose-rich medium and some bacteria that are growing exponentially. This system was represented very simply in Figure 9.13. After a short period of time, you can assume that the system reaches a steady state, and that the cellular concentrations of metabolites won’t change appreciably.

However, the concentrations of metabolites *external* to the cell will change, even at steady state. For example, the amount of glucose in the medium will go down, as will that of other important ions. Other concentrations – for example, the waste products secreted by the cells and even the biomass itself – will increase. These changes in external concentrations are accounted for by using exchange fluxes (Section 9.8).


Figure 10.1 highlights the impact of exchange fluxes on our steady-state assumption.
None of the intracellular metabolite concentrations change over time, but notice that due
to the exchange fluxes, the concentrations of $N_{out}$ and $X_{out}$ can change. In other words,
FBA can’t be used to simulate dynamics or concentrations *inside the cell*, but due to the
exchange fluxes the dynamic concentration changes *outside the cell* can be simulated.

## **Section 10.2. Environmental constraints**

The strategy for modeling these concentration changes turns out to be very similar to
building a numerical integrator like the one we covered in Chapter 5. In that chapter, we
divided the entire simulation into small time steps, then used the values of variables in the previous time step to calculate the values in the next time step via one of several possible
methods (Euler, midpoint, Runge-Kutta).

![Figure 10.1](https://drive.google.com/uc?export=view&id=1D6Ylzt1gqCzPBhVVCWouMl5TOrVRLASK)



> **Figure 10.1. Modeling changes in concentration over time using FBA.** The intracellular metabolites can’t be modeled in this way, but the external concentrations – including the biomass concentration – can, thanks to the exchange fluxes.

Similarly, for our simulations here we will divide the time into steps, and then for each
step we will determine the input conditions, calculate the bounds on the FBA problem
based on those input conditions, run FBA, and use the FBA results to calculate the output
conditions (Figure 10.2).

![Figure 10.2](https://drive.google.com/uc?export=view&id=1SE4n6-6RFj_S4ur6IFrH1b71ce66XsTb)

> **Figure 10.2. Schematic of the overall approach for dynamic FBA.** Notice that the overall idea is similar to numerical integration: evaluating a function at each time step, and using the output of that function to calculate a new value for the variables (see Chapter 5).

Let’s apply this strategy to the system in Figure 10.1. Here, our input conditions are the
concentrations of nutrient and cells, $N_{out}(i)$ and $X_{out}(i)$, where $i$ is the time step we happen
to be on.

Next, we determine the FBA bounds. You can conceptualize these as environmental
constraints; if there are ten *E. coli* cells and one million nearby glucose molecules, the
likely metabolic response will be quite different than if only five glucose molecules are present. The constraint will therefore depend on how much substrate is available, how
many cells there are, and how long the time step is. We write the equation as follows:

> <h3> $
b_{N,\text{in}}(t_i) \leq \frac{N_{\text{out}}(t_i)}{X_{\text{out}}(t_i) \cdot \Delta t}
$

*(Equation 10.1)*


The numerator is the amount of nutrient at time step $i$; dividing by the amount of
biomass, we obtain the amount of nutrient available to each cell. Finally, dividing this
ratio by the time-step length yields the maximum amount of nutrient available to each
cell over the time interval. That maximum amount is the constraint imposed by the
nutrient in the environment.

Notice that $v_{N,in}$ will generally also have a constraint: maximum capacity (Section 9.11).
Moreover, the environmental constraints are time-dependent. This means that when there is plenty of nutrient in the environment relative to the number of cells, the maximum-
capacity constraint on $v_{N,in}$ will be more limiting, and therefore it will be the dominant constraint. When the nutrient in the environment is largely depleted, then the
environmental constraint on $b_{N,in}$ will be more limiting than that of the maximum
capacity.

In either case, both constraints are calculated and used as inputs to the FBA problem.
FBA then returns the complete metabolic flux distribution for the system, including the
values for all of the exchange fluxes. In our example, that would include $b_{N,in}$ and $b_{X,out}$.
These flux values are critical to determining the new values of $N_{out}$ and $X_{out}$.

## <u> **Practice Problem 10.1** </u>

*You perform an experiment with E. coli growing exponentially with glucose as the sole carbon source. Initially, you measure a concentration of 10 mmol/L of glucose and 1 g/L of DCW in the flask.*

*Several hours later, the glucose concentration has decreased to 0.1
mmol/L, while the dry biomass has increased to 10 g/L.
You decide to simulate the metabolic fluxes at each of these times using FBA, with a time step of 1 min. The maximum uptake rate of glucose for this strain of *E. coli* was previously measured at 3 mmol glucose/(g DCW * h).*

*For both time points, determine the environmental constraint imposed by glucose. Which
is more constraining, the environment or the maximum capacity constraint?*

**Solution:** At the initial time point $t_0,$ the environmental constraint can be calculated
using Equation 10.1 as:

> $b_{N,\text{in}}(t_0) \leq \frac{N_{\text{out}}(t_0)}{X_{\text{out}}(t_0) \cdot \Delta t}$  
$\leq \frac{10 \, \text{mmol glucose/L}}{(\text{g DCW/L}) \cdot \text{min}}$  
$\leq 10 \, \frac{\text{mmol glucose}}{\text{g DCW} \cdot \text{min}}$  

*(Equation 10.2)*

At the later time point $t_1$, the environmental constraint will be:

>$b_{N,\text{in}}(t_1) \leq \frac{N_{\text{out}}(t_1)}{X_{\text{out}}(t_1) \cdot \Delta t}$  
$\leq \frac{0.1 \, \text{mmol glucose/L}}{(10 \, \text{g DCW/L}) \cdot \text{min}}$  
$\leq 0.01 \, \frac{\text{mmol glucose}}{\text{g DCW} \cdot \text{min}}$  

*(Equation 10.3)*

The maximum uptake rate is in terms of hours, not minutes, so a conversion is necessary:

> $3 \, \frac{\text{mmol glucose}}{\text{g DCW} \cdot \text{h}} = \frac{3 \, \text{mmol glucose}}{60 \, \text{g DCW} \cdot \text{min}} = 0.05 \, \frac{\text{mmol glucose}}{\text{g DCW} \cdot \text{min}}$  

*(Equation 10.4)*

Comparison of Equations 10.3 and 10.4 shows that when the glucose concentration is
high (at $t_0$), the maximum uptake rate is more constraining, while the environmental
constraint becomes more limiting as the medium becomes depleted of glucose (at $t_1$).

## **Section 10.3. Integration of FBA simulations over time**

With the environmental bounds determined, we are ready to constrain FBA problems.
Now it’s time to learn how the output of FBA can be used to calculate new external
concentrations. Let’s begin with calculating the biomass. I already mentioned in Chapter
7 that the growth of a population of bacteria in the presence of ample nutrients is often
exponential. In particular, it follows this equation:

> <h3> $\frac{dX}{dt} = μX$

*(Equation 10.5)*

the solution of which is:
> <h3> $X(t_{i+1}) = X (t_i)e^{μΔt}$

*(Equation 10.6)*

The constant μ is the growth rate, and as you learned in Chapter 9, the biomass exchange
flux determined by FBA corresponds to the production of new cells; in fact, it is also
equal to the growth rate. So in this case,

> <h3> $μ = b_{X,out}$

*(Equation 10.7)*

Substituting our system in Figure 10.1 and the growth rate from Equation 10.6 into
Equation 10.7 yields:

> <h3> $X_{out} (t_{i+1} = t_i + Δt) = X_{out} (t_i)e^{b_{X ,out}Δt}$

*(Equation 10.8)*

Solving Equation 10.8 for $X_{out}(t_{i+1})$ becomes straightforward because $X_{out}(t_{i+1})$ and $b_{X,out}$ are
already known, and $Δt$ is chosen by the modeler to be just barely long enough for the
steady-state assumption of FBA to hold (a choice of 1-5 s is typical). Notice also that
Equation 10.8 can be used for any FBA problem that has a biomass exchange flux; the
value of the flux, the time step, and the initial concentration of the biomass are sufficient
to calculate the new biomass concentration.

The new value for $N_{out}$ can be determined in a similar manner. In this case, it is useful to
recognize that the flux is simply the change in concentration over a given time step,
normalized by cell biomass. As a result, the ratio of two fluxes at a given time step will
be equal to the ratio of the changes in concentration for that same time step. In our
example, the ratio of the changes in $N_{out}$ and $X_{out}$ is equal to the ratio of the corresponding
exchange fluxes:

> $\frac{N_{\text{out}}(t_i) - N_{\text{out}}(t_{i+1})}{X_{\text{out}}(t_{i+1}) - X_{\text{out}}(t_i)} = \frac{b_{N,\text{in}}}{b_{X,\text{out}}}$  

*(Equation 10.9)*


Rearranging, we obtain:

> $N_{\text{out}}(t_{i+1}) = N_{\text{out}}(t_i) - \frac{b_{N,\text{in}}}{b_{X,\text{out}}} \cdot \left[ X_{\text{out}}(t_{i+1}) - X_{\text{out}}(t_i) \right]$  
$= N_{\text{out}}(t_i) - \frac{b_{N,\text{in}}}{b_{X,\text{out}}} \cdot X_{\text{out}}(t_i) \left( e^{b_{X,\text{out}} \Delta t} - 1 \right)$  

*(Equation 10.10)*

Notice that this formulation depends on the stoichiometric coefficient of the exchange fluxes. In Figure 10.1, we drew the equations such that $b_{N,\text{in}}$ is positive when it enters the system, and $b_{X,\text{out}}$ is positive when it leaves the system. As a result, Equation 10.10 is a difference: more uptake and growth leads to less available nutrient. Equation 10.10 can also be generalized for all extracellular metabolites that have a corresponding exchange flux in the FBA problem. This includes not only substrates, but also by-products; for a by-product, the second term on the right of Equation 10.10 would be added, not
subtracted.

## <u> **Practice Problem 10.2** </u>

*Let’s go back to the experiment and simulation in Practice Problem 10.1. You set the bounds for FBA for the initial time point. Remember from Chapter 9 that the biomass pseudo-flux ($v_{N2X}$ for this system) converts a number of millimoles of various biomass
components into dry cell weight, with units of mmol/g DCW. We assume that 3 mmol of biomass components is converted to 0.01 g DCW.*

Given the above information, FBA returns:

$b_{N,in}$ = 3 mmol/g DCW/h

$v_{N,in}$ = 3 mmol/g DCW/h

$v_{N2X}$ = 0.01/h

$v_{X,out}$ = 0.01/h

$b_{X,out}$ = 0.01/h


where the overall flux is bounded by the maximum capacity constraint from Practice
Problem 10.1.
With this output, 10 mmol/L glucose, and 1 g/L DCW, calculate the glucose and biomass
concentrations at t = 1 min.

**Solution:** First, calculate the biomass from Equation 10.8:

$$
X_{\text{out}} \left(t_0 + \Delta t \right) = X_{\text{out}} \left(t_i \right)e^{b_{X,\text{out}} \Delta t}
$$

$$
X_{\text{out}} \left(t_0 + \frac{1}{60} h \right) = 1 \frac{g}{L} \text{DCW} \ast e^{0.01 \frac{1}{h} \ast \left(\frac{1}{60} \right)h}
$$

$$
= 1.0002 \frac{g}{L} \text{DCW}
$$



Then, calculate the new concentration of nutrient from Equation 10.10:

$$
N_{\text{out}} \left(t_{i+1} \right) = N_{\text{out}} \left(t_i \right) - \frac{b_{N,\text{in}}}{b_{X,\text{out}}} \ast \left[ X_{\text{out}} \left(t_{i+1} \right) - X_{\text{out}} \left(t_i \right) \right]
$$

$$
= 10 \frac{\text{mmol}}{L} - \frac{3 \frac{\text{mmol}}{g \text{DCW} \ast h}}{0.01 \frac{1}{h}} \ast \left[ 1.0002 - 1 \right] \frac{g \text{DCW}}{L}
$$

$$
= \left( 10 - 0.06 \right) \frac{\text{mmol}}{L} = 0.94 \frac{\text{mmol}}{L} \text{glucose}
$$



The outputs we calculated in Practice Problem 10.2 would be equal to the inputs we
would use for the next time step. As I mentioned earlier, this method is very similar to
the numerical integration we learned in Chapter 5. The only real difference is in our use
of FBA to obtain the exchange fluxes that are used to calculate the next time step. And
as we’ve already learned, FBA can solve thousands of equations simultaneously.

## **Section 10.4. Comparing dynamic FBA to experimental data**

One of the earliest demonstrations of using dynamic FBA to simulate cell growth,
substrate uptake, and by-product secretion is illustrated in Figure 10.3. Here, Palsson and
Amit Varma grew *E. coli* on glucose minimal medium (the environment contains glucose
as the sole carbon source, plus a few salts). Under their experimental conditions, *E. coli*
produced biomass and the fermentative byproduct acetate (for now, look at the data from
time = 0-8 h).

![Figure 10.3](https://drive.google.com/uc?export=view&id=1fd7clD0w3rvmFj3kaNdZR4OYSLb-dkIa)

> **Figure 10.3. Dynamic FBA simulation of cell growth in a glucose minimal environment encompasses the changes in glucose concentration (A), cellular biomass (B), and acetate concentration (C). Points represent experimental data.**
Grey boxes highlight the portion of the simulation for which the environmental
constraints substantially impact the simulation. Points that deviate significantly
from FBA predictions are highlighted in red. Modified from Varma, A., Palsson, B.
O. Stoichiometric flux balance models quantitatively predict growth and metabolic
by-product secretion in wild-type Escherichia coli W110. *Applied and
Environmental Microbiology*. 1994. **60**(10): 3724-3731, reproduced/amended with permission from American Society for Microbiology.

These data were also used to simulate these growth conditions. From the glucose data,
Varma and Palsson determined the maximum glucose uptake rate; they obtained the
maximum acetate secretion rate from the acetate data. These two parameters were then
used as maximum capacity constraints in the FBA problem. The overall time course was
divided into small time steps (something on the order of seconds or a minute would
suffice), and the process in Figure 10.2 was carried out for each time step.

During the first part of the simulation, the glucose concentration was high enough that the
environmental conditions were less limiting than the maximum uptake constraints. This scenario lasted for ~6 h, after which the environmental constraints became significant and
increasingly limited the FBA solution (grey in Figure 10.3A, B).

Once the glucose ran out, the cells were left in a pool of acetate that they had secreted,
and the cells were able to reuse that acetate (Figure 10.3C). It’s important to emphasize
that no bounds were changed to produce this simulation; it’s simply that the structure of
the metabolic network cannot accommodate simultaneous uptake of acetate and glucose –
an interesting prediction made by FBA that we also see experimentally! Notice also that
the biomass concentration also continues to increase while acetate is being reutilized, but
not at the same growth rate (Figure 10.3B).

## **Section 10.5. FBA and transcriptional regulation**

As I mentioned, FBA captures the reutilization of acetate, but not perfectly. The dynamic
simulation did not fit the data very well (red points in Figure 10.3C) because an extra
biological process is at work, one that isn’t represented in the FBA framework:
transcriptional regulation of gene expression. Acetate reutilization requires the
expression of certain metabolic genes that are not expressed in the presence of glucose.
Once glucose has been depleted, these enzymes must be synthesized before acetate can
be used, and the process of gene expression and protein synthesis takes several minutes.


It is impossible to fit an accurate line to the acetate data without adding information about
transcriptional regulation. Unfortunately, FBA as we’ve described it here has no
framework to incorporate such regulation. The seriousness of this problem is underscored by the fact that our acetate example involves only a small number of regulated genes; under typical growth conditions in the lab, such as culture in a nutrient-
rich medium, up to ~50% of the genes in E. coli can be down-regulated!


For my doctoral research, I focused on a way to incorporate transcriptional regulation
into FBA. I had already learned about FBA in much the same way you did in Chapter 9,
reading about conservation equations, solution spaces, and optimal solutions, and how
they could be used to simulate metabolic-network behaviors. My next step was to learn
about transcriptional regulation, and I spent about a year and a half in the library, going
through the literature on gene expression in E. coli so that I could identify what was
known and how people represented what they knew. I found that for the most part,
transcriptional regulation was understood at a very qualitative level; figures usually
showed arrows with a “+” or “-” attached to them, and manuscripts rarely contained
detailed parameters.

As a result, I turned to the Boolean modeling approach we described in Chapter 2. This
approach seemed like the answer to all of my problems, as it matched the data well and
could easily be scaled to represent all known transcriptional regulation in *E. coli*. But
how could I integrate it with FBA, which was so different?

## **Section 10.6 Transcriptional regulatory constraints**

The critical insight was to realize that regulation imposes constraints. If the expression of
a certain gene is down-regulated, then the fluxes of the reactions that are catalyzed by the
corresponding enzyme(s) must be constrained. In our Boolean representation, that means
that both the upper and lower bounds must be set to zero. Like our other constraints,
regulation-based constraints can impact the solution space (Figure 10.4) and change the
value of the objective function. If our objective in Figure 10.4 is to maximize v1, then
application of the regulatory constraint has a dramatic difference on our computed output.

![Figure 10.4](https://drive.google.com/uc?export=view&id=13CC_gteuhijn37r36i7107mMxFn4zx2v)

> **Figure 10.4. Regulatory constraints affect the solution space of FBA.** If a
protein’s expression is down-regulated at a given time t, then the corresponding
enzymatic flux $v_1$ is set to zero at that time, changing the shape of the solution
space. Here, consider the solution space to be a cone that includes positive
combinations of fluxes $v_1$, $v_2$, and $v_3$ (left). At some instant in time, an enzyme
associated with the flux of $v_1$ is down-regulated. The $v_1$ flux is therefore
constrained to zero, and as a result, the solution space at that point in time only
contains solutions with zero values for $v_1$ (right).

Regulatory constraints are different from the other constraints we’ve discussed. They are
time-dependent, like environmental constraints, but they are also self-imposed: the cell’s
own machinery imposes the constraint. It might be helpful for you to visualize a large
solution space that encloses the full metabolic network available to E. coli, and then
consider a smaller space that is the solution space at a *particular instant in time*, based on
the dynamic constraints imposed by the environment and by regulation (Figure 10.4).
This smaller space is continually moving around inside of, but never outside of, the larger
space as conditions change.

## **Section 10.7 Regulatory FBA: method**

To integrate, we need to incorporate a new step into our dynamic FBA method, one that
calculates regulatory constraints (Figure 10.5). When I developed this method, I called it
“regulatory FBA” or rFBA for short (see Covert, Schilling, and Palsson in Recommended
Reading).

![Figure 10.5](https://drive.google.com/uc?export=view&id=1n4-wnO2siq-Q9gpV9wq8qhV-pvBuI0pe)

> **Figure 10.5. Schematic of the rFBA approach.** rFBA resembles the approach
illustrated in Figure 10.2, but with a new step (red): the determination of
regulatory constraints based on the evaluation of regulatory rules.

Let’s turn to our glucose uptake/acetate reutilization example for an example. One of the
genes required for acetate reutilization that is down-regulated when glucose is present is
*aceA*, which encodes the isocitrate lyase enzyme *AceA*. The transcription factor that
induces aceA expression is CRP. CRP-based regulation is complicated (Figures 1.1 and
7.1), but for now we’ll just say that CRP is inactive when glucose is present.

We can express the information in the preceding paragraph as Boolean statements. First,
we write a rule for CRP activity based on the extracellular glucose concentration:

> CRP(t) = IF NOT ([Glucose(t)] > 0)

This rule depends only on extracellular glucose concentrations, but you can also write
rules based on the activities of other proteins, or even on the values of fluxes. Next, we
write rules for the expression of aceA; we’ll combine transcription and translation into
one event called “expression”:

> $\text{Expression}_{AceA}$(t) = IF CRP(t)
> AceA(t) = $\text{Expression}_{AceA}$(t) AFTER SOME TIME

These rules would be sufficient for a strictly Boolean model, but since we will be
considering time steps, we need to define “AFTER SOME TIME” more carefully. In
earlier chapters, we talked about how the average gene requires a few minutes to
synthesize mRNA, and then a few more minutes for translation to produce protein.

Assuming that we want a significant amount of AceA to build up in our system, we could
estimate that “AFTER SOME TIME” means after 24 min (the value that best fit the data
in Figure 10.3). Then the above rule becomes:

> AceA(t) = $\text{Expression}_{AceA}$(t) AFTER 24 min

or

> AceA(t) = $\text{Expression}_{AceA}$(t - 24 min)

*(Equation 10.11)*

In other words, at every time step the presence of the AceA protein will depend on
whether or not its expression was initiated 24 min before. Notice that in this model, all of
the delays (the “SOME TIME”s) must be specified (24 min). If the rules determine that
the protein is absent at time t, then the following constraint will be applied:

> <h3> $0 ≤ v_{AceA}(t) ≤ 0$

*(Equation 10.12)*

which is the same as saying that vAceA = 0; this is just like modeling a gene knockout,
except for the dependency on time. If the protein is present at time t, no regulatory
constraint will be applied and the flux $v_{AceA}$ can take on any value. To re-emphasize the
last sentence: the incorporation of regulatory rules does not mean that the fluxes now
only hold Boolean values. The final value of vAceA can hold any real value, including
zero. The Boolean rules only specify whether or not a constraint is applied.

Walking through rFBA (Figure 10.5) is basically the same as for the simulation of
dynamics (Figure 10.2). We begin by defining the rFBA stoichiometric matrix, the
reversibility and capacity constraints, and the objective function. We write all of the gene
expression rules and specify the delay terms. We determine the initial concentrations of
all moieties for which an exchange flux has been defined, including cellular biomass.
Finally, we choose a time step length and then we’re ready to go!

From the initial conditions, you can determine the magnitude of the environmental
constraints, as we discussed for dynamic FBA. You can also calculate the regulatory
state of the cell (the activities of all transcription factors, whether a given gene is being
expressed, and whether the corresponding protein exists) using Boolean modeling. This
regulatory state determines the regulatory constraints on metabolism for that time step.
Then you run rFBA, which yields the flux values for all of the internal (v) and exchange
(b) fluxes, which you use to update the extracellular concentrations of biomass,
substrates, and by-products.

## **Section 10.8. rFBA: application**

This approach simulates glucose uptake and depletion as well as subsequent reutilization
of the acetate byproduct (Figure 10.6A, B). Notice that the curve for acetate reutilization
now agrees beautifully with the data (Figure 10.6C, grey). Importantly, this model also
generates the regulatory network output: for every time point, the activity of all the
transcription factors, as well as expression of all of the genes, proteins, and corresponding
flux bounds, are calculated and stored by the simulation. A few of the most important time points, transcription factors, and genes appear in Figure 10.6D.

![Figure 10.6](https://drive.google.com/uc?export=view&id=1Dd4B4LaHsLpCYnvqEo91YiQBchcIlTb8)

> **Figure 10.6. Adding regulatory information via rFBA results in a match
between experimental data and simulated data.** (A, B) Experimental data (points) and simulations (curves) for changes in the glucose concentration in the
medium (A) and the cellular biomass in the system (B). (C) rFBA (curve) yields
a much better fit to the acetate data (points), particularly during the period of acetate reutilization (grey box). (D) Key features of Oh and Liao’s experimental data (see Recommended Reading). GLU, glucose; AC, acetate. Time points
appear as the column labels. Modified from Covert, M. W., Palsson, B. Ø.
Transcriptional regulation in constraints-based metabolic models of *Escherichia
coli.* *The Journal of Biological Chemistry.* 2002. **277**: 28058-28064,
reproduced/amended with permission of The American Society for Biochemistry
and Molecular Biology.

Interestingly, even this simple model was able to resolve a mystery that had recently
arisen in the literature. At that time, the gene expression microarray had just been
invented. This technology enabled the simultaneous measurement of the expression of
thousands of genes (Sidebar 1.2), providing the first exciting glimpses of whole
transcriptional regulatory networks. James Liao and colleagues at the University of
California, Los Angeles performed one of the earliest studies of this kind in *E. coli*, and
they studied growth on glucose and acetate, the same as our example system (see Oh and
Liao in Recommended Reading). Notice that the experimental measurements are a 100%
qualitative match with the model’s predictions (Figure 10.6D).

Now for the mystery: some of the genes seemed to be shifting for no known reason. As
Oh and Liao wrote:



> Surprisingly, despite the extensive work on E. coli physiology in different carbon
sources, still many genes are regulated for unknown purposes by unknown
mechanisms. For example, several genes were unexpectedly up-regulated, such
as pps [and others] in acetate ... or down-regulated, such as [others] and adhE in
acetate... [emphasis mine].



When I first read this paper in detail, I did a double take because I knew that my model
was predicting the expression changes of these genes correctly. In other words, the
mechanisms were not “unknown,” they were simply complex! All I had to do was
examine the output of my model to see what was causing rFBA to predict this expression.

The answer lay in an interaction between the metabolic and regulatory networks. CRP
switches from OFF to ON in the regulatory network once glucose is depleted, at t = 7 h,
36 min (Figure 10.6D). The switch turns on *aceA, aceB*, and *acs* 24 min later (“AFTER

SOME TIME”). As a result, the metabolic network switches from glycolysis (glucose-
utilizing) to gluconeogenesis (glucose-synthesizing), allowing the cell to use acetate as a

carbon and energy source.

The metabolic switch triggers a shift in the activity of another transcription factor, Cra
(catabolite repressor/activator). Cra turns on at 8 min, 3 s (Fig. 10.6D), and as a result of
Cra activity, *adhE* and other genes are down-regulated and *ppsA* is up-regulated (Figure
10.6D). The model’s capacity to “figure this out” was surprising to *E. coli* experts and
extremely exciting to me at the time.

In the end, rFBA enabled us to make several significant advances in modeling *E. coli*.
First, we were able to build an integrated regulatory-metabolic model that accounted for
1,010 genes: 906 metabolic genes and 104 transcription factors that regulated over half of
those genes (see Covert et al. 2004 in Recommended Reading). We showed that this
model had better predictive capacity (more correct predictions) and a broader scope of
predictions (more environmental and genetic perturbations) than using only the metabolic
network.

## **Section 10.9 Toward whole-cell modeling**

The development of rFBA also turned out to be a major step toward building whole-cell
models, although we didn’t realize it at the time. The idea to separate the total
functionality of the cell into smaller pieces that could be modeled individually and then
integrated together was a critical aspect of our strategy for modeling entire cells (Figure
10.7).

The current whole-cell model of the bacterium *Mycoplasma genitalium* includes 28 sub-
models that are represented using mathematical modeling approaches that can be roughly characterized into eight groups (see Karr et al. in Recommended Reading). We quickly
discovered that the variables constituted a heterogeneous set that needed to be divided.
For example, the number of mRNAs of a particular gene is a very different type of data
than the location on the chromosome that binds a certain DNA repair protein. As a result,
we grouped cell variables into 16 categories.

I admit that Figure 10.7 looks complicated. Don’t let that worry you, because the overall
approach in Figure 10.7 is essentially the same as that in Figures 10.3 and 10.5: the
variable values are used as input to the sub-models, which can be run in series or in
parallel, and the outputs of the sub-models are then fed to the variables. Once again, we
choose short (1-s) time steps and assume that the sub-models operate essentially
independently of each other during that time step, an assumption that renders
computationally feasible a simulation this complex.

A detailed treatment of the whole-cell model would be the subject of another book (the
Karr et al. paper in the Recommended Reading has a ~120-page supplement available
online). However, I’d like to include some details of our model in order to help you tie
everything together.

![Figure 10.7](https://drive.google.com/uc?export=view&id=1mLYm2hZTeuYHLcwnIgQcM2SbP85XsFlH)

> **Figure 10.7. Schematic of the whole-cell modeling process.** The overall approach is the same as in Figures 10.3 and 10.5, but with many more biological processes to model and a very heterogeneous data set that was divided into 16 types of “cell variables.” Note that several types of mathematical models were integrated together. Adapted by permission from MacMillan Publishers Ltd: Gunawardena, J. Silicon dreams of cells into symbols. *Nature Biotechnology*.
Copyright 2012. **30**:838-40.

The model in Figure 10.7 is based on *M. genitalium*, a small bacterial parasite that causes
a sexually transmitted disease in humans; more relevant to our purposes, it is thought to
be the simplest of all culturable bacteria. We selected it as the first cell to model
precisely because it is “simple” and “culturable.” *M. genitalium* has only 525 genes in its
genome, ~1/8 of the number of genes in the *E. coli* genome. We also knew that a critical
part of our efforts would be to test and validate our model experimentally. We scoured
~900 published investigations and databases to gather ~1,700 known biological
parameters. We defined the sub-models and variables in Figure 10.7 based on this
information.

I’ve already indicated that breaking up cellular functionality was critical to the model’s
success, but we implemented a second insight as well: we focused on modeling *one single cell* for *one single cell cycle*. This strategy might at first seem non-intuitive; it
certainly did to me. After all, most data on cells come from populations of cells, not
single cells!

Modeling a single cell cycle provides several advantages. For one thing, you can keep
track of each molecule individually. For example, look at Figure 10.8, a beautiful
painting of a cross-section of *Mycoplasma mycoides*. This image is rendered to scale,
and what you should notice immediately is how crowded everything is. Furthermore,
you can leverage your knowledge of the limited space in a cell to make good estimates of
molecule numbers. For example, let’s examine the ribosomes in Figure 10.8. I count ~20
ribosomes total in the drawing. Considering that this is a cross-section through the
middle, perhaps with a volume of one-fourth to one-eighth of the total volume, we
estimate 80-160 ribosomes in a typical *Mycoplasma* cell (our model uses ~120
ribosomes).

Keeping track of each molecule individually enables several modeling approaches (e.g.,
stochastic simulations). Additionally, modeling a single cell imposes strong constraints
on the parameter set because the size and mass of the cell at the beginning of the cell
cycle must be roughly equal to those of the two daughter cells at the end of the cell cycle.

![Figure 10.8](https://drive.google.com/uc?export=view&id=1BC5nzjsSfCHVAHsLk6FUcmHK6KsE4E8M)

> **Figure 10.8. Painting of Mycoplasma mycoides.** This illustration, which is to
scale, is used with permission from Dr. David S. Goodsell of the Scripps
Research Institute. See
http://mgl.scripps.edu/people/goodsell/illustration/mycoplasma.

The whole-cell model has been incredibly exciting for my lab at Stanford University.
Our simulations compare well with many different types of data. We’ve identified
emergent properties in the model that help us to better understand how cells work.
We’ve followed up on some of the model’s predictions experimentally, and we were
thrilled to see that the model correctly predicted biological parameters that have never
been measured before (see Sanghvi et al. in Recommended Reading).

We are currently trying to make the whole-cell model widely accessible by giving away
the source code and knowledge-bases that we’ve constructed. Additionally, we’re
investigating ways to interrogate the terabytes of data that whole-cell modeling has
produced. For example, we’ve developed WholeCellViz (Figure 10.9), a web-based
application that enables anyone to explore our simulations in detail
(http://wholecellviz.stanford.edu and Karr et al. in Recommended Reading).

But this model is really only the first step into a tantalizing future of whole-cell modeling.
Our model needs to be improved, refined, and expanded. Models of more complex cells
will need to consider issues such as detailed transcriptional regulation (which is generally
absent from *M. genitalium*), compartmentalization (particularly for modeling eukaryotic
cells), and detailed spatial modeling using ODEs. Unexpected hurdles will add even
more spice to the development of whole-cell models, which is currently a very new (and exciting!) field of research. The field needs new scientists with in-depth training at the
intersection of applied mathematics and molecular biology, which is the major reason I
wrote this book. I anticipate that some of you will end up contributing to this effort.

But for now, congratulations! You’ve worked with several powerful mathematical,
numerical, and analytical techniques, and you’ve applied them to many circuits,
networks, and pathways in cells. I hope you have developed some intuition that will
guide you toward determining which methods are useful in particular modeling
situations, as well as a feel for how and when to apply them. Most of all, I hope that this
knowledge serves you well and that you employ it at the cutting edge of science.

![Figure 10.9](https://drive.google.com/uc?export=view&id=1cyrYh3r7FW0j3e0ayODSKFgNJU6lX5BS)

> **Figure 10.9. Visualizing whole-cell modeling data with WholeCellViz.** (A) A
representation of cell shape. (B) Metabolic map, including fluxes and metabolite
concentrations. (C) Map of the M. genitalium chrosome, with information about
polymerization, methylation, and protein binding. (D) FtsZ contractile ring size. (E)
Protein map with information about protein synthesis. (F) Changes in chromosome
superhelicity during the cell cycle. Modified from Lee, R., Karr, J. R., Covert, M. W.
WholeCellViz: data visualization for whole-cell models. *BMC Bioinformatics*. 2013.
**14**(1): 253. This work is licensed under the Creative Commons Attribution License.

## **Chapter Summary**

The last chapter of this book brings together most of the topics that we’ve covered. First,
we discussed how to generate time courses for metabolite uptake and secretion as well as
cell growth using FBA (dynamic FBA). This method depends on the exchange fluxes
calculated by FBA. The exchange fluxes return the growth rate and uptake and secretion
rates, which can be used to determine the changes in biomass and external metabolite
concentrations over time. Additionally, the environment itself can impose constraints on
the FBA model, for example when the availability of nutrients is limited.


The overall process of dynamic FBA resembles the numerical integrators we discussed in
Chapter 5: the value of variables at an initial time point are input into a function that
calculates new variable values as an output. These values are then used as the inputs for
the next time point, and so on. The FBA function is somewhat more complicated than
the functions used for Euler or Runge-Kutta integration, but other than that, the methods
are very similar.


By comparing dynamic FBA model predictions to experimental data (E. coli growing in a
flask containing glucose, leading to glucose depletion and secretion and acetate
reutilization), we uncovered a critical limitation of FBA: the lack of transcriptional
regulatory information. This limitation was addressed by adding regulatory constraints to
FBA (rFBA). Regulatory constraints differ from other constraints in that they are both
self-imposed and time-dependent. The constraints are calculated by evaluating
regulatory rules, using the Boolean logic approach that we developed in Chapter 2. This
process occurs at each time step, and so rFBA is essentially the same as dynamic FBA,
but with the additional step of evaluating regulatory rules.

Finally, rFBA can be seen as a first step toward whole-cell modeling. Like rFBA, whole- cell models bring together multiple modeling approaches into one integrated simulation; like rFBA and dynamic FBA, the process of whole-cell modeling resembles a numerical
integrator. However, whole-cell models include many more cellular processes and variables, and although a whole-cell model has been created for M. genitalium, the field
remains in its infancy.

**Recommended Reading**

* Covert, M. W., Knight, E. M., Reed, J. L., Herrgård, M. J. Palsson, B. Ø. Integrating
high-throughput and computational data elucidates bacterial networks. *Nature*. 2004. **429**(6987): 92-6.

* Covert, M. W., Palsson, B. Ø. Transcriptional regulation in constraints-based metabolic
models of *Escherichia coli. The Journal of Biological Chemistry.* 2002. **277**: 28058-28064.

* Covert, M. W., Schilling, C. H., Palsson, B. Ø. Regulation of gene expression in flux
balance models of metabolism. *Journal of Theoretical Biology.* 2001. **213**(1): 73-88.

* Domach, M. M., Leung, S. K., Cahn, R. E., Cocks, G. G., Shuler, M. L. Computer model for glucose-limited growth of a single cell of *Escherichia coli* B/r-A. *Biotechnology and
Bioengineering.* 1984. **26**(3): 203-16.

* Gunawardena, J. Silicon dreams of cells into symbols. *Nature Biotechnology.* 2012. **30**:838-40.

* Karr, J. R., Sanghvi, J. C., Macklin, D. N., Gutschow, M. V., Jacobs, J. M., Bolival, B.
Jr., Assad-Garcia, N., Glass, J. I., Covert, M. W. A whole-cell computational model
predicts phenotype from genotype. *Cell*. 2012. **150**(2): 389-401.

* Lee, R., Karr, J. R., Covert, M. W. WholeCellViz: data visualization for whole-cell models. *BMC Bioinformatics.* 2013. **14**(1): 253.

* Morowitz, H. J. The completeness of molecular biology. *Israel Journal of Medical Sciences.* 1984. **20:**750-753.

* Oh, M. K., Liao, J. C. Gene expression profiling by DNA microarrays and metabolic
fluxes in Escherichia coli. Biotechnology Progress. 2008. 16(2): 278-86.

* Sanghvi, J. C., Regot, S., Carrasco, S., Karr, J. R., Gutschow, M. V., Bolival, B. Jr.,
Covert, M. W. Accelerated discovery via a whole-cell model. Nature Methods. 2013.
10(12): 1192-5.

* Savinell, J. M., Palsson, B. Ø. Network analysis of intermediary metabolism using linear
optimization. I. Development of mathematical formalism. Journal of Theoretical
Biology. 1992. 154(4): 421-54.

* Shuler, M. L., Foley, P., Atlas, J. Modeling a minimal cell. Methods in Molecular
Biology. 2012. 861: 573-610.

* Tomita, M., Hashimoto, K., Takahashi, K., Shimizu, T. S., Matsuzaki, Y., Miyoshi, F.,
Saito, K., Tanida, S., Yugi, K., Venter, J. C., Hutchison, C. A. 3rd. E-CELL: software
environment for whole-cell simulation. Bioinformatics. 1999. 15(1):72-84.

* Wade, N. “Life is Pared to Basics; Complex Issues Arise.” The New York Times 14
December 1999.

## **Problems**

### **Problem 10.1. Dynamic FBA concepts**

Let’s say you want to make a flux balance model for the growth of *E. coli* on ribose. A
simplified metabolic network is shown below. Notice that two isomerases, encoded by
*rpiA* and *rpiB*, exist in *E. coli* for the interconversion of ribulose 5-phosphate and ribose
5- phosphate. Either of these isomerases can catalyze both the forward and the backward
reactions, with fluxes *$v_{rpiA}$* and *$v_{rpiB}$*, respectively. X represents the cellular biomass
(Figure 10.1). Note also the exchange fluxes for ribose, ATP, ADP, and X.

![Figure 10.1 hw](https://drive.google.com/uc?export=view&id=1J40814l_vdU8Crl9qOvXfHn4M3cvSQM7)

**(a)** Write the flux-balance equations in matrix form. Your answer will include the
stoichiometric matrix. Note also that $\text{ribose}_{out}$ and $X_{out}$ should not be included in this formulation, as the exchange fluxes are for $X_{in}$ and ribosein (don’t worry, they
will show up later).

**(b)** Let the initial amount of external ribose be 10 mmol/L, the initial amount of
biomass be (1/e) g DCW/L (where e is the base of the natural logarithm), and
assume that there is more than enough ATP and ADP available (no environmental
constraints due to ATP or ADP). Transport of ribose into the system is limited
such that 0 < bin < 1 mmol/g DCW/min, and the maximum throughput for $v_{rpiA}$
has been measured at 0.5 mmol/g DCW/min. All other bounds are infinite, with
the exception of the thermodynamic constraints that are indicated in the diagram.
Our objective is to maximize the output of biomass at each time step. Working by
hand, calculate the output of the first time step that would be produced using
dynamic FBA (including $b_{rib}$, $b_X$, [$\text{ribose}_{in}$], [$X_{out}$], and the sum ($v_{rpiA}$ + $v_{rpiB}$)). For
this problem, assume that 1 mmol of ribulose-5-phosphate is made per 1 g DCW
of biomass, so that the units of $b_X$ will be 1/min. Use a time step of 1 min.

**(c)** The expression of rpiA is thought to be constitutive; however, expression of rpiB
occurs only in the absence of the repressor RpiR, which in turn is active only in
the absence of external ribose. Assume a time delay of 5 time steps for the
amount of protein to reflect changes in transcription. Write the regulatory rules for the system. How would you incorporate this information into your answer to (b)? What changes would you expect to see in your calculations?

**(d)** You would like to generate some double knockout strains for experiments.
Typically, double knockouts are made sequentially; one knockout strain is made
first, and serves as the platform strain for the second knockout. Which of the
following mutants are viable (can produce X) in the presence of glucose as the
only carbon source? An *rpiA* knockout; an *rpiB* knockout; an *rpiR* knockout; an
*rpiA* knockout followed by an *rpiR* knockout; an *rpiR* knockout, followed by an *rpiA* knockout. You may assume that the ribose phosphate isomerase reaction
must be catalyzed to produce biomass.

### **Problem 10.2. Implementing dynamic FBA**

This problem deals with glycolysis, the conversion of one molecule of glucose to two
pyruvate molecules through multiple enzymatic conversion steps. The external source of
glucose can come from glucose itself or from other molecules such as lactose, one
molecule of which can be broken down into a glucose and a galactose molecule by the
*lacZ* gene product:

![Figure 10.2 hw](https://drive.google.com/uc?export=view&id=1RWmJV36wSBqvx8ga_g4f45AROKeF1KwD)

**(a)** Construct a stoichiometric matrix that describes all of the chemical reactions in
this system. Let the rows represent metabolites and the columns represent
enzymes. It may be easier to create the stoichiometric matrix in Excel and import the data into Python.

**(b)** Add exchange fluxes to your stoichiometric matrix to represent the input/output
of extracellular glucose and lactose, ATP, ADP, P$_i$, NAD$^+$, NADH+, H+, and H$_2$O to and from the system. Assume that the objective of the network is to maximize pyruvate output. Create an exchange flux b$_{pyruvate}$ for this purpose.

**(c)** Load your stoichiometric matrix into Python. If you built your matrix in Excel, you can use the Python function `xlsread`. Set upper and lower bounds based on the reaction reversibility shown in the diagram. Let the upper bounds through the pts and lacY transport reactions be 10 mmol/g DCW/h and 5 mmol/g DCW/h, respectively. What does your flux solution look like? Does the cell primarily utilize lactose or glucose catabolism? Graph the value of each flux in v (I used `barh` for this).

**(d)** We will now extend your FBA model of glycolysis in order to produce a dynamic
simulation. Use the following information:

$G_0$ = initial concentration of glucose = 10 mmol/L

$L_0$ = initial concentration of lactose = 10 mmol/L

$X_0$ = initial concentration of dry cellular biomass = 0.1 g DCW/L

$dt$ = length of the time step = 0.05 h

You may also assume that all other cofactors are present in excess, and that production of 100 mmol of pyruvate corresponds to the production of 1 g DCW of cellular biomass. Plot the concentrations of glucose, lactose, and dry cellular
biomass in the medium over 75 h. Describe your observations.

### **Problem 10.3. rFBA**

Now we will explore the effects of transcriptional regulation on the results of our FBA
simulation in Problem 10.2. Recall that the *E. coli* *lac* operon (Figure 7.1) encodes three
genes: *lacZ* (encodes β-galactosidase, which cleaves lactose into glucose and galactose),
*lacY* (the permease that transports lactose into the cell), and *lacA* (a transacetylase). The
operon is controlled primarily by the *lac* repressor LacI, which prevents transcription of
the operon in the absence of lactose, and the glucose-regulated activator CRP. CRP is
active in the absence of glucose and LacI is active in the absence of lactose.

We will consider the following regulatory scheme: if the glucose level in the medium is ≥
0.1 mmol/L, CRP is inactive. Once the glucose level falls below 0.1 mmol/L, CRP is
activated. Additionally, if the lactose level in the medium is ≥0.1 mmol/L, LacI is
inactive. Once the lactose level falls below 0.1 mmol/L, LacI is active. Fifteen minutes
after transcription of *lacY* and *lacZ* is initiated, *LacY* and β-galactosidase are produced
(with the same upper and lower bounds as in Problem 10.2).

**(a)** Write out the rules for your system, and explain qualitatively what you expect to
see.

**(b)** Adjust your code from your dynamic FBA in Problem 10.2 to include this
transcriptional regulation as a time-dependent constraint on your FBA problem.
Note: there will be a period early in the situation (before 15 min of simulation
time) in which you will not be able to evaluate the rule for Lac protein expression;
in this case, simply use the initial value of *lac* gene transcription at time zero.
Plot the activities of CRP and LacI, as well as the transcription and presence of
LacY and β-galactosidase over 75 h. Explain your plots.

**(c)** Plot the concentrations of glucose, lactose, and cellular biomass over time. Compare these plots to those you generated as part of Problem 10.2d. Why do you suppose that the lac genes are regulated to produce this behavior?